In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
from selenium import webdriver
from datetime import date
from datetime import date,datetime,timedelta
import time as t
import re
import sys
# connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
client = MongoClient('localhost', 27017)
try :
    db = client['indeed2']
except :
    db = client.indeed2
indeed = db['indeed6']




#liste de toutes les querys dans la bare de recherche indeed
#######
all_query_poste = ['développeur','business intelligence',
                   'data scientist','data science','data analyst','data engineer','data architect',
                   'data protection','data manager','data concepteur','big data','devops',
                   'web developer','full stack developer','web architect','front end developer','back end developer'] #

all_query_ville = ['Nantes','Bordeaux','Île-de-France','Lyon', 'Toulouse']

all_query_contrat = ['permanent','fulltime','subcontract','internship',
                     'apprenticeship','contract','custom_1','temporary','parttime']
#######

#lance une boucle sur l'algo de scrapping pour tout nos query 
#######
try :
    for query_poste in all_query_poste :
        for query_ville in all_query_ville:
            for query_contrat in all_query_contrat:
                lien = 'https://www.indeed.fr/emplois?q=' + query_poste + '&l=' + query_ville + '&jt=' + query_contrat
                browser = webdriver.Firefox(executable_path='geckodriver.exe')
                browser.get(lien)
                browser.maximize_window()
                #change url try
                scraploop(browser,query_poste,query_ville,query_contrat,indeed)
#######

#pour avoir un retour sur plusieurs info durant la situation de l'erreur
#######
except Exception as e: 
    print('2Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
    print('query_poste ' + query_poste) 
    print('\n query_ville ' + query_ville)
    print('\n all_query_contrat ' + query_contrat)
#######

test
3Error on line 136 WebDriverException Message: Failed to decode response from marionette

2Error on line 47 InvalidSessionIdException Message: Tried to run command without establishing a connection

query_poste développeur

 query_ville Nantes

 all_query_contrat permanent


In [1]:
#mail_pop_up quitte l'onglet qui propose d'envoyer notre mail qui pop up dans certaines situations.
def mail_pop_up():
    try :
        CLICK = WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR , '#popover-close-link')))
        CLICK.click()
        t.sleep(0.2)
    except :
        pass

#scraploop scrap toutes les pages (1 à 100 pages max) obtenues depuis les query decidé au-dessus.
def scraploop(browser,query_poste,query_ville,query_contrat,indeed,time = 0):
    #retien le nom de notre page dans le cas où on as plusieur onglet
    window_name = browser.window_handles[0]
    row = {}
    
    #quitte le pop_up constant àla 2nd page 
    if time == 1 :
        print('test')
        mail_pop_up()
    time += 1
    
    #cherche toutes les offre d'emploit sur la page
    try:
        post_info =  WebDriverWait(browser, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME , 'title')))
    
    #lance la prochaine query si il y a rien
    except :
        browser.close()
        return
    
    #cherche toutes les ID des offre d'emploit sur la page
    ID = browser.find_elements_by_class_name('jobsearch-SerpJobCard')
    
    #boucle qui scrap toutes les offres de la pages et les implémente dans notre bdd mongodb
    for i,post in enumerate(post_info) : 
        post.location_once_scrolled_into_view
        post.click()
        
        #sleep au cas où (car très rare) quand on clique sur un offre ca nous ouvre les info sur un autre onglet
        t.sleep(0.3)
        
        #Si ouvre nouvelle onglet, on la quitte et relance la page
        if len(browser.window_handles) >1 :
            print('hi browser.window_handles :)')
            browser.switch_to.window(window_name=browser.window_handles[1])
            browser.close()
            t.sleep(0.2)
            browser.switch_to.window(window_name=window_name)
            browser.refresh()
            mail_pop_up()
            scraploop(browser,query_poste,query_ville,query_contrat,indeed,time)
        
        #extraction de toutes les données
        row['query_poste'] = query_poste
        row['query_contrat'] = query_contrat
        row['query_ville'] = query_ville
        row['_id'] = ID[i].get_attribute("id")
        row['titre'] = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID , 'vjs-jobtitle'))).text
        row['nom_de_la_boite'] =  browser.find_element_by_id('vjs-cn').text
        row['adresse'] =  browser.find_element_by_id('vjs-loc').text
        row['texte'] =  browser.find_element_by_css_selector('#vjs-content').text



        try :
            iswhat = browser.find_element_by_css_selector('div.jobMetadataHeader-itemWithIcon:nth-child(2)').text
            if any(char.isdigit() for char in iswhat) != True :
                row['contrat'] =  iswhat
                try :  
                    row['salaire'] =  browser.find_element_by_css_selector('div.jobMetadataHeader-itemWithIcon:nth-child(3) > span:nth-child(2)').text  #vjs-jobinfo > div:nth-child(3)
                except :
                    row['salaire'] = ' '
                    pass
            else :
                row['salaire'] =  iswhat
        except Exception as e:
            row['salaire'] = ' '
            pass
        try :
            date_text = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR , '#vjs-footer'))).text
            date_text = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR , '#vjs-footer'))).text
            date_value = re.findall(r'\d+|jour|mois|heure', date_text)
            if date_value[1] in 'mois' :
                row['date'] = (datetime.today() - timedelta(days= (int(date_value[0])*30.5))).strftime("%d/%m/%Y, %H:%M:%S")
            elif date_value[1] in 'minutes' :
                row['date'] = (datetime.today() - timedelta(minutes=int(date_value[0]))).strftime("%d/%m/%Y, %H:%M:%S")
            elif date_value[1] in 'jours' :
                row['date'] = (datetime.today() - timedelta(days=int(date_value[0]))).strftime("%d/%m/%Y, %H:%M:%S")
            elif date_value[1] in 'heures' :
                row['date'] = (datetime.today() - timedelta(hours=int(date_value[0]))).strftime("%d/%m/%Y, %H:%M:%S")
        except :
            pass
        try : 
            row['lien_plus_info'] = browser.find_element_by_css_selector('.ws_label').get_attribute("href")
        except :
            pass

        #si l'_id est unique, crée une nouvelle row, si non implémente les infos qu'on a en plus pour cette _id
        try:
            indeed.insert_one(row)
        except :
            #faudrais être plus claire sur les moments de modification (cas où 2/3 choses change en même temps)
            #si modification
            if row['query_poste'] not in indeed.find({'_id':row['_id']})[0]['query_poste']:
                indeed.update({'_id': row['_id']},
                               {"$set" : {
                                           'query_poste': row['query_poste']+', '+indeed.find({'_id':row['_id']})[0]['query_poste']
                                      }
                                }
                            )
            if row['query_contrat'] not in indeed.find({'_id':row['_id']})[0]['query_contrat']:
                indeed.update({'_id': row['_id']},
                               {"$set" : {
                                           'query_contrat': row['query_contrat']+', '+indeed.find({'_id':row['_id']})[0]['query_contrat']
                                      }
                                }
                            )
            if row['query_ville'] not in indeed.find({'_id':row['_id']})[0]['query_ville']:
                indeed.update({'_id': row['_id']},
                               {"$set" : {
                                           'query_ville': row['query_ville']+', '+indeed.find({'_id':row['_id']})[0]['query_ville']
                                      }
                                }
                            )
            if row['salaire'] != ' ' and row['salaire'] not in indeed.find({'_id':row['_id']})[0]['salaire']:
                print(indeed.find({'_id':row['_id']})[0]['salaire'])
                indeed.update({'_id': row['_id']},
                               {"$set" : {
                                           'salaire': row['salaire']+', '+indeed.find({'_id':row['_id']})[0]['salaire']
                                      }
                                }
                            )
    
    #vas à la prochaine page, si pas posible vas au prochain query, car plus de pages 
    Next = browser.find_elements_by_css_selector('.np')
    try :
        Next[0].click() if ( time == 1)  else Next[1].click()
        return scraploop(browser,query_poste,query_ville,query_contrat,indeed,time)
    except Exception as e: 
        print('3Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
        browser.close()
        return